In [1]:
import torch
import torch.nn as nn
import torch.utils.data as utildata
import torchvision.datasets as ds
import torchvision.transforms as transforms
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 디바이스 선택과 데이터셋 init
---
torch.device -> 사용할 device 선택하도록

ds.MNIST -> MNIST dataset을 Tensor 타입으로 받으며 없으면 다운로드

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# init MNIST dataset
trainSet = ds.MNIST(root='./data/', # 
                    train=True,
                    transform=transforms.ToTensor(),
                    download=True)

testSet = ds.MNIST(root='./data/',
                   train=False,
                   transform=transforms.ToTensor(),
                   download=True)
print(trainSet.train_data.size())
# 60000개나 들어있다.
print(testSet.train_data.size())
# 여긴 10000개 batch는 대충 20? 100? 200?

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


#Hypoer parameter
---
일종의 튜닝 옵션

train_epochs -> 에포크는 학습 반복 횟수

learning_rate -> 기울기 찾을때 이동하는 속도

batch_size -> 한번에 불러올 데이터(weight 갱신 주기?)

classes -> MNIST는 0~9이므로 10개다.


In [0]:
# hyper param
train_epochs = 10 # train 몇번 할거냐
learning_rate = 0.001 # 옵티마이저에서 iteration 마다 이동하는 속도?? 뭐라고 해야하지 쨌든 너무 크면 위로 날아가고 너무 작으면 최소 로스 찾기전에 끝남
batch_size = 20 # 전체 데이터에서 일정 수의 샘플로 나눠서 weight 갱신 (여기선 20번마다)
classes = 10 # 0~9

# 데이터셋 불러오는 부분
---
dataset -> 불러올 데이터셋

batch_size -> 위 설명대로

shuffle -> training할때 마다 데이터가 새로 배열된다. 특정 data에만 종속되어 학습되게하는 overfitting을 피할 수 있다고 함

num_workers -> 서브 프로세스 얼마나 돌릴건지

In [0]:
# data loader
train = utildata.DataLoader(dataset=trainSet, # 불러올 데이타셋
                           batch_size=batch_size,
                           shuffle=True, # 매 epoch마다 데이터가 재배열 -> 매 epoch마다 dataset이 섞이기 때문에 overfitting을 피할 수 있음
                           num_workers=4) # 데이터 불러올때 서브 프로세스 몇개 돌릴거냐? 0은 메인 프로세스에서만
test = utildata.DataLoader(dataset=testSet,
                           batch_size=batch_size,
                           shuffle=False,
                           num_workers=4)

# DNN 모델 생성
---
기본적으로 torch.nn 내에 있는 모듈을 상속받아 서브클래스를 만들어 사용한다.

__init__에서 사용할 멤버변수를 forward에서는 정방향으로 연산한다.

Linear는 말 그대로 선형 함수

in_feature는 입력값 out_features는 출력값

bias는 실제 결과값과 얼마나 차이나는지

relu는 activation 함수



In [0]:
# Deep NN Midel
class myNN(nn.Module):
  def __init__(self, classes = 10):
    super(myNN,self).__init__()
    self.layer1 = nn.Linear(in_features=784, out_features=512, bias=True)
    self.layer2 = nn.Linear(in_features=512, out_features=512, bias=True)
    self.layer3 = nn.Linear(in_features=512, out_features=512, bias=True)
    self.layer4 = nn.Linear(in_features=512, out_features=512, bias=True)
    self.layer5 = nn.Linear(in_features=512, out_features=256, bias=True)
    self.layer6 = nn.Linear(in_features=256, out_features=128, bias=True)
    self.layer7 = nn.Linear(in_features=128, out_features=64, bias=True)
    self.layer8 = nn.Linear(in_features=64, out_features=10, bias=True)
  def forward(self, input):
    res = nn.functional.relu(self.layer1(input))
    res = nn.functional.relu(self.layer2(res))
    res = nn.functional.relu(self.layer3(res))
    res = nn.functional.relu(self.layer4(res))
    res = nn.functional.relu(self.layer5(res))
    res = nn.functional.relu(self.layer6(res))
    res = nn.functional.relu(self.layer7(res))
    res = nn.functional.relu(self.layer8(res))
    return res

# Optimizer
---
이제 어떻게 최적의 방향으로 갈건지 정하는 방향이다.

실제 label과 predict값이 얼마나 차이가 나느냐를 loss라고 하는데 이 loss를 최소화 하기 위해 어떠한 방향으로? 갈건지 정하는 알고리즘?

In [6]:
model = myNN(classes).to(device)
print(model)
# optimizer (Gradient Descent, Stochastic Gradientt Descent,
# Adaptive Gradient, Adaptive Delta, Adaptive Moment Estimation ... )
sgd = torch.optim.SGD(model.parameters(), lr=learning_rate)
adagrad = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
adadelta = torch.optim.Adadelta(model.parameters(), lr=learning_rate)
adam = torch.optim.Adam(model.parameters(), lr=learning_rate)

myNN(
  (layer1): Linear(in_features=784, out_features=512, bias=True)
  (layer2): Linear(in_features=512, out_features=512, bias=True)
  (layer3): Linear(in_features=512, out_features=512, bias=True)
  (layer4): Linear(in_features=512, out_features=512, bias=True)
  (layer5): Linear(in_features=512, out_features=256, bias=True)
  (layer6): Linear(in_features=256, out_features=128, bias=True)
  (layer7): Linear(in_features=128, out_features=64, bias=True)
  (layer8): Linear(in_features=64, out_features=10, bias=True)
)


# loss function
---
위에서 loss에 대해 간단하게 말했는데, 이 loss를 구하는 식

crossEntropy가 어떻게 하는거지 까먹음

In [0]:
# loss function (negative log-likelihood)
crossEn = nn.CrossEntropyLoss()

# Train
---
실제로 학습하는 부분이다. 지금까지 위에서 정의한거 다 쓰는곳임
(DNN의 경우 258초 경과)

In [8]:
import time

st = time.time()

for epoch in range(train_epochs):
  for i, (images,labels) in enumerate(train):
    images = images.view(-1,784)
    images = images.to(device)
    labels = labels.to(device)
    adam.zero_grad()
    predic = model(images)
    loss = crossEn(predic,labels)
    loss.backward()
    adam.step()
    
    if(i+1)%100 == 0:
      print('Epoch: [{}/{}], Loss: {:.4f}, Step: {}'.format(epoch+1, train_epochs,loss.item(),i+1))
      
print("--- %s seconds ---" %(time.time() - st))

Epoch: [1/10], Loss: 1.6004, Step: 100
Epoch: [1/10], Loss: 1.4333, Step: 200
Epoch: [1/10], Loss: 0.9616, Step: 300
Epoch: [1/10], Loss: 0.7867, Step: 400
Epoch: [1/10], Loss: 0.0606, Step: 500
Epoch: [1/10], Loss: 0.4446, Step: 600
Epoch: [1/10], Loss: 0.7448, Step: 700
Epoch: [1/10], Loss: 0.3014, Step: 800
Epoch: [1/10], Loss: 0.2522, Step: 900
Epoch: [1/10], Loss: 0.6456, Step: 1000
Epoch: [1/10], Loss: 0.5762, Step: 1100
Epoch: [1/10], Loss: 0.1469, Step: 1200
Epoch: [1/10], Loss: 0.0156, Step: 1300
Epoch: [1/10], Loss: 0.1279, Step: 1400
Epoch: [1/10], Loss: 0.3504, Step: 1500
Epoch: [1/10], Loss: 0.1709, Step: 1600
Epoch: [1/10], Loss: 0.1462, Step: 1700
Epoch: [1/10], Loss: 0.1004, Step: 1800
Epoch: [1/10], Loss: 0.2127, Step: 1900
Epoch: [1/10], Loss: 0.4412, Step: 2000
Epoch: [1/10], Loss: 0.3583, Step: 2100
Epoch: [1/10], Loss: 0.2674, Step: 2200
Epoch: [1/10], Loss: 0.1557, Step: 2300
Epoch: [1/10], Loss: 0.1973, Step: 2400
Epoch: [1/10], Loss: 0.5835, Step: 2500
Epoch: [1

In [0]:
torch.save(model.state_dict(), './gdrive/My Drive/myNN.pkl')

# Test
---
이제 얼마나 잘 학습했는지 test를 하면 된다. (Accr : 97.49)

In [12]:
#model.load_state_dict(torch.load('myNN.pkl'))

model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test:
      images = images.view(-1,784)
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  print('Accuracy test on the 10000 images : {}'.format(100 * correct/total))

Accuracy test on the 10000 images : 97.49
